In [102]:
from pymongo import MongoClient
import pandas as pd
import geopandas as gpd
import json

import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

## Importing MongoDB information:

In [38]:
client = MongoClient("localhost:27017")

In [39]:
db = client.get_database("lab-mongo-queries")

In [40]:
companies = db.get_collection("companies")

In [41]:
companies.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

#### Now we filter down every company with offices in Spain:

In [42]:
filter_spain = {"offices.country_code": "ESP"}
proj = {"_id": 0, "name": 1, "category_code": 1, "offices.$": 1, "funding_rounds": 1}

In [43]:
spain_offices = list(companies.find(filter_spain, proj))

In [44]:
spain_offices[0].keys()

dict_keys(['name', 'category_code', 'funding_rounds', 'offices'])

In [45]:
len(spain_offices)

218

#### Only 218 remain.

In [111]:
spain_offices[0]["offices"]

[{'description': 'Google Madrid',
  'address1': 'Torre Picasso',
  'address2': 'Plaza Pablo Ruiz Picasso 1',
  'zip_code': '28020',
  'city': 'Madrid',
  'state_code': None,
  'country_code': 'ESP',
  'latitude': None,
  'longitude': None}]

#### We make dataframe with each office longitude, latitude, raised amount and category code:

In [112]:
final_offices = []

for doc in spain_offices:
    name = doc["name"]
    category_code = doc["category_code"]
    count = 0
    
    for funds in doc["funding_rounds"]:
        try:
            count += funds["raised_amount"]
        except:
            None
    
    for of in doc["offices"]:
        lat = of["latitude"]
        lon = of["longitude"]
        
        dic = {"name": name, "category_code": category_code, "raised_amount": count, "location": {"type": "Point", "coordinates": [lat, lon]}}
        dic.update(of)
        final_offices.append(dic)

In [113]:
final_offices

[{'name': 'Google',
  'category_code': 'search',
  'raised_amount': 555182792,
  'location': {'type': 'Point', 'coordinates': [None, None]},
  'description': 'Google Madrid',
  'address1': 'Torre Picasso',
  'address2': 'Plaza Pablo Ruiz Picasso 1',
  'zip_code': '28020',
  'city': 'Madrid',
  'state_code': None,
  'country_code': 'ESP',
  'latitude': None,
  'longitude': None},
 {'name': 'Truphone',
  'category_code': 'mobile',
  'raised_amount': 0,
  'location': {'type': 'Point', 'coordinates': [None, None]},
  'description': 'Tru Spain',
  'address1': 'Gran Via, 6. 4th floor',
  'address2': '',
  'zip_code': '28013',
  'city': 'Madrid',
  'state_code': None,
  'country_code': 'ESP',
  'latitude': None,
  'longitude': None},
 {'name': 'Brightcove',
  'category_code': 'enterprise',
  'raised_amount': 103100000,
  'location': {'type': 'Point', 'coordinates': [None, None]},
  'description': '',
  'address1': 'Avenida Diagonal 640',
  'address2': 'planta 6',
  'zip_code': '08017',
  'cit

## Working with the new DataFrame:

In [114]:
spain_df = pd.DataFrame(final_offices)

In [115]:
spain_df.head()

,name,category_code,raised_amount,location,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,Google,search,555182792,"{'type': 'Point', 'coordinates': [None, None]}",Google Madrid,Torre Picasso,Plaza Pablo Ruiz Picasso 1,28020,Madrid,None,ESP,NaN,NaN
1,Truphone,mobile,0,"{'type': 'Point', 'coordinates': [None, None]}",Tru Spain,"Gran Via, 6. 4th floor",,28013,Madrid,None,ESP,NaN,NaN
2,Brightcove,enterprise,103100000,"{'type': 'Point', 'coordinates': [None, None]}",,Avenida Diagonal 640,planta 6,08017,Barcelona,None,ESP,NaN,NaN
3,Negonation,web,0,"{'type': 'Point', 'coordinates': [40.463667, -...",None,,,,,None,ESP,40.463667,-3.74922
4,TokBox,messaging,26000000,"{'type': 'Point', 'coordinates': [None, None]}",Barcelona Office,D00 BUILDING - P14-L11,"PL.Ernest Lluch i Martin, 5",,"Barcelona, 08019",None,ESP,NaN,NaN


#### Let's clean the nulls:

In [116]:


spain_df = spain_df[spain_df["latitude"].isnull() == False]

In [117]:
# We drop the "state_code" column:

spain_df = spain_df.drop("state_code", axis = 1)

In [118]:
spain_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129 entries, 3 to 216
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           129 non-null    object 
 1   category_code  128 non-null    object 
 2   raised_amount  129 non-null    int64  
 3   location       129 non-null    object 
 4   description    121 non-null    object 
 5   address1       129 non-null    object 
 6   address2       129 non-null    object 
 7   zip_code       129 non-null    object 
 8   city           129 non-null    object 
 9   country_code   129 non-null    object 
 10  latitude       129 non-null    float64
 11  longitude      129 non-null    float64
dtypes: float64(2), int64(1), object(9)
memory usage: 13.1+ KB


In [119]:
spain_df = spain_df.reset_index(drop = True)

In [122]:
spain_df.head()

,name,category_code,raised_amount,location,description,address1,address2,zip_code,city,country_code,latitude,longitude
0,Negonation,web,0,"{'type': 'Point', 'coordinates': [40.463667, -...",None,,,,,ESP,40.463667,-3.749220
1,Fon,mobile,44000000,"{'type': 'Point', 'coordinates': [40.530724, -...",,Calle Quintanavides 15,2-1,28050,Madrid,ESP,40.530724,-3.632687
2,eyeOS,enterprise,1000000,"{'type': 'Point', 'coordinates': [41.3854855, ...",HQ,La Rambla 140 4C,,08002,Barcelona,ESP,41.385486,2.170287
3,]project-open[,software,0,"{'type': 'Point', 'coordinates': [41.383182, 2...",,"Ronda de Sant Antonã, 51, 1o 2a",,08011,Barcelona,ESP,41.383182,2.163697
4,Starfish Community Group,advertising,0,"{'type': 'Point', 'coordinates': [41.387917, 2...",,"Major de Sarria 11, 2-1",,08017,Barcelona,ESP,41.387917,2.169919


## Now let's see each offices in a map with Folium:

In [124]:
inicial_lat = 40.60514229875771
inicial_long = -4.429541222343291

In [125]:
map_1 = folium.Map(location= [inicial_lat, inicial_long], zoom_start=6)
map_1

In [126]:
for i, row in spain_gdf.iterrows():
    loc = {
        "location":  [row["latitude"], row["longitude"]],
        "tooltip": row["name"]
    }
    
    icon = Icon(
    color="blue",
    prefix="fa",
    icon="briefcase",
    icon_color="black"
    )
    
    Marker(**loc, icon=icon).add_to(map_1)

In [131]:
map_1

### And let's export it as a JSON file:

In [132]:
spain_df.to_json("data/spain_companies.json", orient="records")